In [2]:
import numpy as np

# Définition de J
def J(mu, x, rho, K):
    return np.dot(mu, x) - 0.5 * np.dot(np.dot(rho * K, x), x)

# Définition du gradient de J par rapport à x
def gradient_J(mu, x, rho, K):
    return mu - rho * np.dot(K, x)

# Fonction pour maximisé J en fonction de x
def max_J_en_x(mu, rho, K, epsilon, N=1e6):
    x_k = np.random.rand(len(K))
    k = 0 
    x_k_prev = np.zeros_like(x_k) # Vecteur de 0 de même taille que x_0 fournit
    
    while np.linalg.norm(x_k - x_k_prev) >= epsilon * np.linalg.norm(x_k_prev): #Condition de sortie

        # Utilisation du simplexe
        x_tilde_k_1 = np.zeros_like(x_k)
        max_lambda = np.argmin(gradient_J(mu, x_k, rho, K))
        x_tilde_k_1[max_lambda] = 1
        
        pas_opti = 2 / (2 + k)  # formule du pas optimisé
        x_k = (1 - pas_opti) * x_k + pas_opti * x_tilde_k_1
         
        k += 1
        x_k_prev = x_k.copy()

        if k > N: # Pour éviter une boucle infini
            break
    
    return x_k

# Fonction pour minimisé H en fonction de mu
def min_H_en_mu(K, rho, epsilon, N=1e6):
    mu_k = np.random.rand(len(K))
    k = 0  
    mu_k_prev = np.zeros_like(mu_k) # Vecteur de 0 de même taille que mu_0 fournit
    
    while np.linalg.norm(mu_k - mu_k_prev) < epsilon * np.linalg.norm(mu_k_prev): #Condition de sortie
        grad_H = max_J_en_x(mu_k, rho, K, epsilon)
        
        # Utilisation du simplexe
        mu_tilde_k_1 = np.zeros_like(mu_k)
        max_index = np.argmin(grad_H)
        mu_tilde_k_1[max_index] = 1
        
        
        pas_opti = 2 / (2 + k)  # formule du pas optimisé
        mu_k = (1 - pas_opti) * mu_k + pas_opti * mu_tilde_k_1
        
        
        k += 1
        mu_k_prev = mu_k.copy()

        if k > N: # Pour éviter une boucle infini
            break
    
    return mu_k


# Exemple d'utilisation
n = 5
mu = np.random.rand(n) # On crée un vecteur mu aléatoire

# On crée une matrice symétrique définie positive K
def SDP_matrix(epsilon=1e-8):
    matrix = np.random.rand(n,n) # On crée une matrice de taille n x n aléatoire
    matrix = np.dot(matrix, matrix.T) # On rend la matrice K symétrique
    eigenvalues, eigenvectors = np.linalg.eigh(matrix) # On calcule les valeurs propres et les vecteurs propres
    eigenvalues[eigenvalues < 0] = epsilon # On rend les valeurs propres non négatives
    positive_definite_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T)) # On reconstruit la matrice avec les vp et vect propre
    return positive_definite_matrix

K=SDP_matrix()

rho = 1
epsilon = 1e-6 

x_solution = max_J_en_x(mu, rho, K, epsilon)
print("x maximisé:", x_solution)


mu_solution = min_H_en_mu(K, rho, epsilon)
print("mu minimisé:", mu_solution)

x maximisé: [0. 1. 0. 0. 0.]
mu minimisé: [0.13545492 0.61709843 0.95097715 0.76610093 0.80776526]
